In [1]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/coref-model-2018.02.05.tar.gz")

/Users/frederikwarburg/anaconda3/envs/neuralcoref/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [7]:
import pandas as pd
import numpy as np
from utils import get_cluster_num,get_span_from_offset, spans
import spacy

In [3]:
data = pd.read_csv('data/gap/gap-test.tsv', sep = '\t')
data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [23]:
#text = 'This is the first sentence. This is the second sentence.'
prediction = predictor.predict(
  document=text
)
prediction['clusters']

Encountered the antecedent_indices key in the model's return dictionary which couldn't be split by the batch size. Key will be ignored.


[[[1, 1], [8, 8], [12, 12]],
 [[19, 20], [39, 40]],
 [[26, 28], [57, 59]],
 [[57, 58], [61, 61], [65, 65], [69, 69]]]

In [ ]:
results = []
for index in range(len(data['Text'])):
    
    text = data['Text'][index]
    prediction = predictor.predict(
      document=text
    )

    pronoun_span = get_span_from_offset(text, data['Pronoun-offset'][index])
    A_span = get_span_from_offset(text, data['A-offset'][index])
    B_span = get_span_from_offset(text, data['B-offset'][index])

    pronoun_cluster_num = get_cluster_num(pronoun_span, prediction)
    A_cluster_num = get_cluster_num(A_span, prediction)
    B_cluster_num = get_cluster_num(B_span, prediction)

    A, B, N = 0, 0, 0
    for pronoun_pred in pronoun_cluster_num:
        if pronoun_pred in A_cluster_num: A = 1 
        if pronoun_pred in B_cluster_num: B = 1 

    if A == 0 and B == 0: N = 1
    
    results.append([A, B, N])
    
    if index % 10 == 0:
        print(index, " / ", len(data['Text']))


0  /  2000
10  /  2000
20  /  2000
30  /  2000
40  /  2000


In [ ]:
results